In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub

In [4]:
(train_examples, valid_examples, test_examples), info = tfds.load('cats_vs_dogs',split=['train[80%:]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True, 
    as_supervised=True )
num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes
print(num_examples)
print(num_classes)

23262
2


In [5]:
#Resizing the image
def format_image(image, label):
    image = tf.image.resize(image, (224,224))/255.0
    return image, label

In [6]:
BATCH_SIZE = 32

In [7]:
#Image Augmentation
#Shuffling, resizing etc batches of images
train_data = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
valid_data = valid_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_data = test_examples.map(format_image).batch(1)

In [11]:
for image_batch, label_batch in train_data.take(1):
    pass
image_batch.shape
print(image_batch[0])

tf.Tensor(
[[[0.25390384 0.24606068 0.24998225]
  [0.25490198 0.24705882 0.2509804 ]
  [0.25892973 0.2510866  0.25500816]
  ...
  [0.47583157 0.4954394  0.5072041 ]
  [0.46798843 0.48759627 0.49936098]
  [0.46798843 0.48759627 0.49936098]]

 [[0.25490198 0.24705882 0.2509804 ]
  [0.25823492 0.25039178 0.25431335]
  [0.2627451  0.25490198 0.25882354]
  ...
  [0.47847515 0.498083   0.50984764]
  [0.47063202 0.49023986 0.5020045 ]
  [0.47063202 0.49023986 0.5020045 ]]

 [[0.25758928 0.24974614 0.2536677 ]
  [0.26110294 0.2532598  0.25718138]
  [0.26296106 0.25511792 0.2590395 ]
  ...
  [0.48504025 0.50464815 0.51641285]
  [0.4771971  0.49680495 0.5085697 ]
  [0.4771971  0.49680495 0.5085697 ]]

 ...

 [[0.00784314 0.00784314 0.00784314]
  [0.00784314 0.00784314 0.00784314]
  [0.00784314 0.00784314 0.00784314]
  ...
  [0.23570842 0.20104451 0.18049134]
  [0.2220639  0.19461292 0.16324036]
  [0.26738426 0.23993327 0.20856072]]

 [[0.00784314 0.00784314 0.00784314]
  [0.00784314 0.00784314 0

# Defining the model
All it takes is to put a linear classifier on top of the feature_extractor_layer with the Hub module.

For speed, we start out with a non-trainable feature_extractor_layer, but you can also enable fine-tuning for greater accuracy.

In [12]:
do_fine_tuning = False

# Load TFHub Module

In [13]:
module_selection = ("mobilenet_v2", 224, 1280) #@param ["(\"mobilenet_v2\", 224, 1280)", "(\"inception_v3\", 299, 2048)"] {type:"raw", allow-input: true}
handle_base, pixels, FV_SIZE = module_selection
MODULE_HANDLE ="https://tfhub.dev/google/tf2-preview/{}/feature_vector/4".format(handle_base)
IMAGE_SIZE = (pixels, pixels)
print("Using {} with input size {} and output dimension {}".format(
  MODULE_HANDLE, IMAGE_SIZE, FV_SIZE))

Using https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4 with input size (224, 224) and output dimension 1280


In [15]:
MODULE_HANDLE ="https://tfhub.dev/google/tf2-preview/{}/feature_vector/4".format(handle_base)
feature_extractor = hub.keras_layer(MODULE_HANDLE, input_shape=IMAGE_SIZE + (3,), 
                                   output_shape=[FV_SIZE],
                                   trainable=do_fine_tuning)

TypeError: 'module' object is not callable